In [26]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, IFrame
from ipywidgets import interact, fixed
import pandas as pd
from numpy import cos,sin,tan,arctan,exp,log,pi,array,linspace,arange
from numpy.linalg import norm
from mpl_toolkits import mplot3d

plt.rcParams["figure.figsize"] = [7,7]

%matplotlib inline

# Uncomment the one that corresponds to your Jupyter theme
plt.style.use('dark_background')
# plt.style.use('fivethirtyeight')
# plt.style.use('Solarize_Light2')

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\bv}[1]{\begin{bmatrix} #1 \end{bmatrix}}$
$\renewcommand{\vec}{\mathbf}$


# One-minute Review

A space curve can represent the **position** of a body at time $t$. $$\vec r(t) = x(t)\vec i + y(t) \vec j + z(t) \vec k.$$ 

Its derivative $$\vec r'(t) = x'(t)\vec i + y'(t) \vec j + z'(t) \vec k$$ is the **velocity vector**. Its magnitude is the **speed**.

The second derivative $\vec r''(t)$ is the **acceleration**. 

In [37]:
r1 = lambda t: np.array([t,t**2,t**3])
r2 = lambda t: np.array([cos(2*pi*t),sin(2*pi*t),t])
r3 = lambda t: np.array([cos(8*pi*t),sin(3*pi*t),sin(7*pi*t)])

@interact
def _(t=(-1,1,.05),r={"twist":r1, "helix": r2, "wacky": r3},angle=(-90,120,6),vangle=(0,90,6),osculating_plane=False
     ):
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111,projection='3d')
    s = np.linspace(-1,1,300)
    ax.view_init(vangle, angle)
    ax.set_autoscale_on(True)
    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    X,Y,Z = np.column_stack([r(x) for x in s])
    # make axes lines
    ax.plot([-1,1],[0,0],[0,0],'k')
    ax.plot([0,0],[-1,1],[0,0],'k')
    ax.plot([0,0],[0,0],[-1,1],'k')
    ax.plot(X,Y,Z,alpha=1,lw=3);
    ax.quiver(0,0,0,r(t)[0],r(t)[1],r(t)[2])
    v = (r(t+.004) - r(t-.004))*250
    v = v/norm(v) # scale for visibility only
    ax.quiver(r(t)[0],r(t)[1],r(t)[2],v[0],v[1],v[2],color="green")
    a = (r(t+.004) - r(t)*2 + r(t-.004))*250**2
    a = a/norm(a) # scale for visibility only
    ax.quiver(r(t)[0],r(t)[1],r(t)[2],a[0],a[1],a[2],color="red")
    if osculating_plane:
#         verts=[r(t) + v + a,r(t) + v - a,r(t) - v - a]
#         ax.add_collection3d(Poly3DCollection(verts,alpha=.3))
#         verts=[r(t) + v + a,r(t) - v - a,r(t) - v + a]
#         ax.add_collection3d(Poly3DCollection(verts,alpha=.3))
        xs,ys,zs = np.column_stack([r(t) + v + a,r(t) + v - a,r(t) - v + a,r(t) - v - a])
        ax.plot_surface(xs.reshape(2,2),ys.reshape(2,2),zs.reshape(2,2),alpha=.3,zorder=.6)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, min=-1.0, step=0.05), Dropdown(descript…

<p style="padding-bottom:40%;"> </p>

# Lecture 06

  - Objectives

    - Describe motion of objects using calculus of curves.
    - Compute arc length of curves.
    - Explain the quantity $\kappa$, the curvature.
  
  - Resources
    - Content
      - Stewart: §13.3-4
      - New Strang: [§3.3](https://cnx.org/contents/oxzXkyFi@5.30:9fEOYsBF@9/3-3-Arc-Length-and-Curvature) [§3.4]()
    - Visualization
      - [CalcPlot3D](https://www.monroecc.edu/faculty/paulseeburger/calcnsf/CalcPlot3D/)
    - Practice
      - Mooculus: [Motion and Paths in Space](https://ximera.osu.edu/mooculus/calculus3/motionAndPathsInSpace/titlePage)
    - Extras
      - CalcBLUE: [Vector Calculus and Motion](https://www.youtube.com/watch?v=rq4kxVFnOUA&index=48&list=PL8erL0pXF3JYm7VaTdKDaWc8Q3FuP8Sa7)

### Motion Example

Find the angle $\alpha$ relative to the ground that maximizes the distance of an object thrown with speed $v_0$. 

<p style="padding-bottom:40%;"> </p>

In [28]:
# range of a projectile launched from the ground with angle alpha and speed v ft/sec

@interact
def _(alpha = (0,pi/2),v=(0,100)):
    x=np.linspace(0,10,101)
    plt.figure(figsize=(7,7))
    plt.plot(v*cos(alpha)*x,v*sin(alpha)*x-32*x**2/2)
    plt.ylim([0,160])
    plt.xlim([0,350])

interactive(children=(FloatSlider(value=0.7853981633974483, description='alpha', max=1.5707963267948966), IntS…

# Arc Length

Most have an intuition for what is meant by the length of a curving path. But carefully formulating it gives rise rather quickly to a fundamental formula.

In [22]:
r = lambda t: np.array([cos(2*pi*t),sin(2*pi*t),sin(6*pi*t)])

@interact
def _(n=(1,100,5),angle=(-30,120,6)):
    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(111,projection='3d')
    s = np.linspace(0,1,300)
    ax.view_init(30, angle)
    ax.set_autoscale_on(True)
    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    X,Y,Z = np.column_stack([r(x) for x in s])
    ax.plot(X,Y,Z,alpha=1,lw=3);
    s = np.linspace(0,1,n+1)
    X,Y,Z = np.column_stack([r(x) for x in s])
    ax.plot(X,Y,Z,alpha=1,lw=3);
    # make axes lines
    ax.plot([-1,1],[0,0],[0,0],'k')
    ax.plot([0,0],[-1,1],[0,0],'k')
    ax.plot([0,0],[0,0],[-1,1],'k')


interactive(children=(IntSlider(value=46, description='n', min=1, step=5), IntSlider(value=42, description='an…

## Arc Length

So a reasonable approximation is 

$$\sum_{i=1}^n |\vec r(a + i \Delta t) - \vec r(a+(i-1)\Delta t)|$$ 

$$ = \sum_{i=1}^n \left|\frac{\vec r(a + i \Delta t) - \vec r(a+(i-1)\Delta t)}{\Delta t}\right| \Delta t $$ since $\Delta t >0$.

$$ \longrightarrow \int_a^b |\vec r'(t)|\,dt$$


<p style="padding-bottom:40%;"> </p>

### Example

Find the arc length of one coil of the helix $$\cos t\,\vec i + \sin t\,\vec j + t\,\vec k.$$

In [33]:
@interact(unroll=(pi/50,2*pi,pi/50),paper=(0,.6,.1))
def _(unroll=pi/50,paper=0,angle=(-96,96,6)):
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111,projection='3d')
    t = arange(0,unroll+pi/100,pi/100)
    ax.plot(np.ones_like(t),t,t,color='b');
    x,y=np.meshgrid(t,linspace(0,2*pi,3))
    ax.plot_surface(np.ones_like(x),x,y,zorder=-1,alpha=paper,color='orange')
    t = arange(0,2*pi-unroll+pi/100,pi/100)
    ax.plot(cos(t),unroll + sin(t),t+unroll,color='b');
    x,y=np.meshgrid(t,linspace(0,2*pi,3))
    if unroll < 2*pi:
        ax.plot_surface(cos(x),unroll*np.ones_like(x) + sin(x),y,zorder=-1,alpha=paper,color='orange')
    ax.set_ylim((-1,2*pi-1))
    ax.set_xlim((-2*pi+1,1))
    ax.view_init(30,angle)

interactive(children=(FloatSlider(value=0.06283185307179587, description='unroll', max=6.283185307179586, min=…

<p style="padding-bottom:40%;"> </p>

## Reparametrization

The same path in space can be traced in any number of ways—fast, slow, backwards. The process of changing this is called **reparametrization**. In effect, we replace the argument to a curve $\vec r(t)$ with a different scalar-valued function to make $\vec r(f(t))$.

#### Quick example

$\langle \cos t, \sin t\rangle$ traces out the unit circle over the interval $[0,2\pi]$.

$\langle \cos (16\pi t), \sin (16\pi t)\rangle$ traces out the unit circle much "faster", over the interval $[0,\frac18]$.

Take the derivatives of each of the above to see this.

<p style="padding-bottom:40%;"> </p>

### Parametrization by arc length

Let $\vec r(t)$ for $t\in[a,b]$ be a smooth (i.e., $|\vec r'(t)| \neq 0$) curve. Let $$s(t) = \int_a^t |\vec r'(\tau)|\,d\tau$$ be the arc length function. Note:

  - $L = s(b)$ is the length of the curve. 

  - $\frac{ds}{dt} = |\vec r'(t)|$.

  - $s(t)$ is monotonically increasing, thus invertible. We write $\vec r(s)$ for $s\in [0,L]$ to be this curve **parametrized by arc length**.

  - $\vec r'(t) = \frac{d\vec r}{ds} \frac{ds}{dt}$

  - $\vec T = \frac{d\vec r}{ds} =\frac{\vec r'(t)}{|\vec r'(t)|}$ is a unit vector. 

### Example

Parametrize the curve $$\langle e^{-t}, 1-2e^{-t}, 2e^{-t} \rangle$$ for $t \geq 0$ by arc length. 

<p style="padding-bottom:40%;"> </p>

## Curvature

Which of these curves is "curviest"?

In [24]:
r1 = lambda t: np.array([t,2*t,-t])
r2 = lambda t: np.array([t*cos(2*pi*t),t*sin(2*pi*t),t/4])
r3 = lambda t: np.array([cos(pi*t),sin(pi*t),t])

@interact
def _(angle=(-90,120,6),vangle=(0,90,6)):
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(111,projection='3d')
    s = np.linspace(-1,1,300)
    ax.view_init(vangle, angle)
    ax.set_autoscale_on(True)
    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    # make axes lines
    ax.plot([-1,1],[0,0],[0,0],'k')
    ax.plot([0,0],[-1,1],[0,0],'k')
    ax.plot([0,0],[0,0],[-1,1],'k')
    for i,r in enumerate((r1,r2,r3)):
        X,Y,Z = np.column_stack([r(x) for x in s])
        ax.plot(X,Y,Z,alpha=1,color='byr'[i],lw=3,label=f"curve {i+1}");
#     v,w =[r(t),(r(t+.01)-r(t-.01))/.07]
    ax.legend()


interactive(children=(IntSlider(value=12, description='angle', max=120, min=-90, step=6), IntSlider(value=42, …

<p style="padding-bottom:40%;"> </p>

#### Definition of curvature

Recall that $\vec T$ is the **unit tangent vector** to a curve $\vec r(t)$. The **curvature** $\kappa$ of $\vec r$ at a given point is given by $$\kappa = \left|\frac{d\vec T}{ds} \right|.$$

##### Formula

Perhaps a more useful formula comes from $\vec T'(t) = \frac{d\vec T}{ds} \frac{ds}{dt} $, or 

$$\kappa = \frac{|\vec T'(t)|}{|\vec r'(t)|}.$$

<p style="padding-bottom:40%;"> </p>

### Example

Find the curvature of a circle of radius $a$. 

In [35]:
@interact(a=(1,10,.3),s=(-1,1,.01))
def _(a=1,s=0):
    t = linspace(-pi,pi,500)
    plt.figure(figsize=(7,7))
    plt.plot(a*cos(t/a),a*sin(t/a))
    plt.xticks(arange(-1,a+1.5,.25))
    plt.yticks(arange(-2,2,.25))
    plt.quiver(a*cos(s/a),a*sin(s/a),-sin(s/a),cos(s/a),angles='xy', scale_units='xy', scale=1,color='red')
    plt.xlim((a-2,a+1))
    plt.ylim((-1.5,1.5))
    plt.grid(True)

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, min=1.0, step=0.3), FloatSlider(value=…